# Búsqueda de parametros ideales para KNN y KNN+PCA.

#### En este notebook buscamos cual es la cantidad k de vecinos ideal para el algoritmo de KNN. Luego buscamos cual es el par ideal de k vecinos y alpha componentes principales para el algoritmo de KNN+PCA.

## Compilando el módulo desde C++

#### Compilamos el modulo desde C++. En CMakeList.txt está seteado que se corra con OpenMP para paralelizar operaciones

In [1]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Found libomp without any special flags
-- pybind11 v2.6.3 dev1
-- Found PythonInterp: /home/orga2/tp2/bin/python (found version "3.6.9") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/lib

In [2]:
# Importamos el tp. Importamos funciones.
import metnum
import funciones
#Armamos el dataset
X,Y,X_test,Y_test = funciones.construir_dataset()
X_trains,Y_trains,X_vals,Y_vals = funciones.get_KFold_sets(X,Y)

/home/orga2/Metodos/metnum-1c2021-tp2/notebooks
Python 3.6.9


## Buscamos obtener el par k, aplha que maximice la performance de este clasificador. Luego, con los parametros obtenidos, lo compararemos con el clasificador sin pesos.

### Comenzamos calculando el accuracy para los intervalos [1,15] de k y [20,35] de alpha. Esperamos quelos parametros ideales de este clasificador sean cercanos a los del knn clasico.

In [11]:
tupla, tiempo = funciones.encontrarParOptimo(X_trains,Y_trains,X_vals,Y_vals,conPeso=True,heatMap=True)
paramPesos = (tupla[0],tupla[1])
paramKNN = (8,35) #le tire lo que me acuerdo pero aca va el valor calculado en parametrosIdeales

100%|██████████| 15/15 [1:16:27<00:00, 305.83s/it]


### Ahora compararemos su desempeño y calcularemos la kappa de cohen

In [ ]:
pcaPesos = metnum.PCA(paramPesos[1])
pcaKNN = metnum.PCA(paramKNN[1])
clfPesos = metnum.KNNClassifier(paramPesos[0],conPeso = True)
clfKNN = metnum.KNNClassifier(paramKNN[0],conPeso = False)

pcaPesos.fit(X)
pcaKNN.fit(X)
XPesos = pcaPesos.transform(X)
XKNN = pcaKNN.transform(X)
X_testPesos = pcaPesos.transform(X_test)
X_testKNN = pcaKNN.transform(X_test)

clfPesos.fit(Xpesos)
clfKNN.fit(XKNN)
y_predPesos = clfPesos.predict(X_testPesos)
y_predKNN = clfKNN.predict(X_testKNN)

accPesos = sk.metrics.accuracy_score(y_predPesos, Y_test)
accKNN = sk.metrics.accuracy_score(y_predKNN, Y_test)
cohen = sk.metrics.cohen_kappa_score(y_predPesos,y_predKNN)